[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/drive/run.ipynb)

In [ ]:
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2
!pip install -q --pre xformers
!pip install -q --pre triton

%cd /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui
!git reset --hard
!git pull

!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-civitai-browser pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-webui-additional-networks pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/sd-webui-controlnet pull
!git -C /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/extensions/openpose-editor pull

!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/launch.py
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/launch.py

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
!sed -i '/import warnings/a from gradio import strings' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/modules/ui.py
!sed -i "/from gradio import strings/a strings.en[\"PUBLIC_SHARE_TRUE\"] = f\"WebUI Colab URL: ${webui_url}\"" /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/modules/ui.py

!python launch.py --xformers --enable-insecure-extension-access --gradio-queue